In [1]:
from validate_adrresses import validate_address
import pandas as pd
import os
from tqdm import tqdm
print(os.getcwd())

/home/luis/Documents/Belcorp/iteration_adresses


In [2]:
df = pd.read_csv('./data/direcciones.csv', sep=';')

/tmp/ipykernel_109457/4095483860.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/direcciones.csv', sep=';')


In [3]:
cols = [col for col in df]
col1 = cols[:3]
col2 = cols[3:]

cols = []
for col in col1:
    cols.append(col)
cols.append("NUEVA DIRECCION")
for col in col2:
    cols.append(col)

In [4]:
df["NUEVA DIRECCION"] = None
df = df.reindex(columns=cols)
df = df[df["IND TERRITORIOS"] == 9]

In [5]:
df_tmp = df[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO", "NUEVA DIRECCION", "X", "Y"]]
df_tmp

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
4,"MARIA VELIZ ALCAYAGA, VALLENAR, ATACAMA, CHILE...",VALLENAR,None,,
8,UNO 11 POSTE 669700 MINA CARACOLES RINCONADA,SAN ESTEBAN,None,,
11,PJE. QUILLAY 1485 POBLACION EL BOSQUE DE OCALI...,ANGOL,None,,
13,LOS CLAVELES 66.ERCILLA.LA ARAUCANIA. VILLA MI...,ERCILLA,None,,
16,EMA QUIROGA LAGUNA VERDE,VALPARAISO,None,,
...,...,...,...,...,...
33051,ARTURO PRAT 510 CASA DOS PISOS,SAN ANTONIO,None,-38.7471312,-72.9524371
33067,ISABEL RODAS POBLACION,VALDIVIA,None,,
33069,LORD COCHRANE 611 CERRO ARENAS,SAN ANTONIO,None,,
33070,EL ARADO 504 VILLA OBLONERA,LA FLORIDA,None,,


In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

# Assuming df is your original Pandas DataFrame
ddata = dd.from_pandas(df_tmp, npartitions=30)

# Sample meta dataframe to indicate the structure of the output
meta = pd.DataFrame({
    "DIRECCION DE DOMICILIO": pd.Series(dtype='str'),
    "COMUNA DOMICILIO": pd.Series(dtype='str'),
    "NUEVA DIRECCION": pd.Series(dtype='str'),
    "X": pd.Series(dtype='float'),
    "Y": pd.Series(dtype='float')
})

def apply_validate_address(df_chunk):
    # Handle missing values before passing to validate_address
    df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]] = df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]].fillna("")
    
    # Apply the validation function to each row and unpack the results
    df_chunk[["NUEVA DIRECCION", "X", "Y"]] = df_chunk.apply(
        lambda row: pd.Series(validate_address(row["DIRECCION DE DOMICILIO"] + " " + row["COMUNA DOMICILIO"])),
        axis=1
    )
    return df_chunk

# Map the function to each partition, with the meta keyword to define the structure
res = ddata.map_partitions(apply_validate_address, meta=meta).compute()

Obispo Villarroel, Cerro Perdices, Playa Ancha, Valparaíso, Provincia de Valparaíso, Región de Valparaíso, 2370541, Chile -33.0360234 -71.6380764
Pasaje Julia Molina, Don Matías 2, Teno, Provincia de Curicó, Región del Maule, Chile -34.8723778 -71.157932
Pedro Aguirre Cerda, Rere, Yumbel, Provincia de Bío-Bío, Región del Biobío, Chile -37.134751 -72.7266698
Policía de Investigaciones de Chile, 977, Presidente Federico Errázuriz, Cerro de la Cruz, Punta Arenas, Provincia de Magallanes, Región de Magallanes y de la Antártica Chilena, 6201065, Chile -53.1653036 -70.9070226
sin coincidencias
Padre Las Casas, Provincia de Cautín, Región de la Araucanía, Chile -38.7674245 -72.5956312
Aurora de Chile, Barrio Aurora de Chile, Recoleta, Provincia de Santiago, Región Metropolitana de Santiago, 7690000, Chile -33.41393 -70.6280934
El Agro, Negrete, Provincia de Bío-Bío, Región del Biobío, 4540000, Chile -37.6123744 -72.5740153
sin coincidencias
Las Quilas, Condominio Las Quilas, Temuco, Provincia

In [11]:
res

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
4,"MARIA VELIZ ALCAYAGA, VALLENAR, ATACAMA, CHILE...",VALLENAR,None,NaN,NaN
8,UNO 11 POSTE 669700 MINA CARACOLES RINCONADA,SAN ESTEBAN,"Ruta E-821, Hacienda Rinconada de Los Andes, E...",-32.800571,-70.715278
11,PJE. QUILLAY 1485 POBLACION EL BOSQUE DE OCALI...,ANGOL,"Pasaje, Villa El Vaticano, Callejones, San Vic...",-34.442319,-71.088176
13,LOS CLAVELES 66.ERCILLA.LA ARAUCANIA. VILLA MI...,ERCILLA,None,NaN,NaN
16,EMA QUIROGA LAGUNA VERDE,VALPARAISO,None,NaN,NaN
18,PASAJE LOMA CORTADA SN.CURARREHUE.LA ARAUCANIA...,CURARREHUE,"300, Pasaje Santa Sofía, Comunidad Huichacura ...",-38.777519,-72.588531
21,CHACABUCO 3662 POBLACION GUSTAVO LEPAIGE,CALAMA,None,NaN,NaN
27,"AURORA 9108 DEPTO 21 A, PRIMER BLOCK ROJO, FRE...",PUDAHUEL,"Calle Jaime Guzmán Errázuriz, Lomas del Estrec...",-53.183965,-70.930572
34,VICTOR JARA 11358 DPTO 201 LOTEO B,LA PINTANA,"Chile, 23 de Agosto, Loteo Apaza, Municipio de...",-24.385761,-65.272172
37,CAMPAMENTO GENESIS 2 CASA A48 LUIS CRUZ MARTIN...,ANTOFAGASTA,"Rua Chile, Vila Souza, Mirassol, Região Imedia...",-20.814314,-49.500920


In [ ]:
print("A")